In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data=pd.read_csv("/content/historic_demand_2009_2024.csv",parse_dates=[1])

In [ ]:
data.head()

Explotary Data Analysis

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
# formatting date columns

In [ ]:
data=data.rename(columns={'settlement_date':'date'})

In [ ]:
data.head()

In [ ]:
data['date']=pd.DataFrame(data['date'])

In [ ]:
data['date']=pd.to_datetime(data['date'])

In [ ]:
data['date']=data['date'].dt.date

In [ ]:
data.head()

In [ ]:
data1=data.groupby(['date'])[['nd', 'tsd', 'england_wales_demand','embedded_wind_generation', 'embedded_wind_capacity',
       'embedded_solar_generation', 'embedded_solar_capacity', 'non_bm_stor',
       'pump_storage_pumping','is_holiday']].mean()
data.index=data['date']
data1.asfreq('D')
data.head()

In [ ]:
data1.tail(10)

In [ ]:
data1.describe()

In [ ]:
import calendar

In [ ]:
lab=['National ','Transmission System ','ENGLAND_WALES']
co=['darkviolet','tomato','darkorange']
data.columns

In [ ]:
data1.head()

In [ ]:
data1.index=pd.to_datetime(data1.index)

In [ ]:
data1.index.dtype

In [ ]:
fig,axs=plt.subplots(3,1,figsize=(30,14),sharex=False)
for i,j,k in zip(data1.columns,range(0,3),lab):
    sns.boxplot(data=data1,x=data1.index.month,y=i,ax=axs[j],palette='Set3',saturation=1)
    axs[j].set_xticklabels([z for z in calendar.month_abbr[1:]],rotation=30,fontsize=15,)
    axs[j].set_title(f'Electricity Demand: {k}',fontsize=20,fontweight='bold')
plt.tight_layout()
plt.show()

we can see variation in jun jul and august are same


In [ ]:
f,axs=plt.subplots(3,1,figsize=(20,12),sharex=False)
for i,j,l,k in zip(data1.columns,[0,1,2], ['orangered','dodgerblue','violet'],lab):
        sns.histplot(data=data1,bins=40,x=i,ax=axs[j],kde=True,stat='density',color=l)
        axs[j].set_title(f'Electricity Demand :{k}',fontsize=20,fontweight='bold')
plt.tight_layout()
plt.show()

All the distribution are right skewed

In [ ]:
fig,axs=plt.subplots(3,1,figsize=(20,10))
ys=data1.columns       #i         #j       #k                                  #l
for i,j,l,k in zip(data1.columns,[0,1,2], ['orangered','dodgerblue','violet'],lab):
    sns.lineplot(data=data1,x=data1.index,y=i,ax=axs[j],label=k,color=l,linewidth=1.6)
    axs[j].set_title(f'  Electricity Demand :  {k}',fontsize=18)
    axs[j].set_ylabel('Mega Watts')
plt.tight_layout()
plt.title('Electricity Demand Line Chart')
plt.show()

From here we can see that the demand of the electricty consumption is decreasing slightly in the last years which is the good thing for us it might be because of public awarness programs and literacy.There is presence of some seasonality and trends in the data

In [ ]:
data1['2023-01-01':][['nd','tsd','england_wales_demand']].plot(figsize=(20,5),linewidth=3,
                                        title='Endogeneous Variables performances till now form 2023 ',cmap='brg',ylabel='MW')

At the end of the december the the demand is high this might be due to the christmas and again in the end of the janaury it is peaking again

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Lambda, Dense, LSTM, SimpleRNN, Dropout,GRU,Bidirectional,LeakyReLU,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD,Adam,Adagrad
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping,ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler as mms
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from tensorflow.keras.models import load_model

In [ ]:
scaler=MinMaxScaler()
data_scaled=scaler.fit_transform(data1)

In [ ]:
data_scaled

In [ ]:
features=data_scaled

In [ ]:
target=data_scaled[:,0]

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [ ]:
TimeseriesGenerator(features,target,length=100,batch_size=100)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(features,target,test_size=0.2,shuffle=False)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
win_length=100
batch_size=32
num_feature=10
train_gen=TimeseriesGenerator(x_train,y_train,length=win_length,batch_size=batch_size)
test_gen=TimeseriesGenerator(x_test,y_test,length=win_length,batch_size=batch_size)

In [ ]:
train_gen[0]

In [ ]:
model=Sequential()
model.add(LSTM(units=128,return_sequences=True,input_shape=(win_length,num_feature)))
model.add(LSTM(units=64,return_sequences=True,activation='relu'))
model.add(LSTM(units=32,return_sequences=True,activation='relu'))
model.add(LSTM(units=16,return_sequences=True,activation='relu'))
model.add(LSTM(units=8,return_sequences=True,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(54,activation='relu'))
model.add(Dense(28,activation='relu'))
model.add(Dense(12,activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(units=1,activation='linear'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=1,mode='min')

In [103]:
history=model.fit(train_gen,epochs=100,validation_data=test_gen,shuffle=False,callbacks=[early_stopping])

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


139/139 ━━━━━━━━━━━━━━━━━━━━ 71s 418ms/step - loss: 0.1759 - mse: 0.1759 - val_loss: 0.0189 - val_mse: 0.0189
Epoch 2/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 60s 428ms/step - loss: 0.0332 - mse: 0.0332 - val_loss: 0.0187 - val_mse: 0.0187
Epoch 3/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 76s 389ms/step - loss: 0.0331 - mse: 0.0331 - val_loss: 0.0153 - val_mse: 0.0153
Epoch 4/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 56s 403ms/step - loss: 0.0297 - mse: 0.0297 - val_loss: 0.0154 - val_mse: 0.0154
Epoch 5/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 52s 378ms/step - loss: 0.0267 - mse: 0.0267 - val_loss: 0.0152 - val_mse: 0.0152
Epoch 6/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 54s 392ms/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.0166 - val_mse: 0.0166
Epoch 7/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 60s 433ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.0147 - val_mse: 0.0147
Epoch 8/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 53s 385ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.0153 - val_mse: 0.0153
Epoch 9/100
139/139 ━━━━━━━━━━━━━━━━

In [104]:
model.evaluate(test_gen)

33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 0.0135 - mse: 0.0135


[0.015091937966644764, 0.015091938897967339]

In [105]:
prediction=model.predict(test_gen)

33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 174ms/step


In [106]:
prediction.shape[0]

1037

In [107]:
prediction

array([[[0.4008428 ],
        [0.40340886],
        [0.4010173 ],
        ...,
        [0.34378618],
        [0.34543777],
        [0.347181  ]],

       [[0.40078974],
        [0.40293524],
        [0.39919046],
        ...,
        [0.34543845],
        [0.3471816 ],
        [0.34906244]],

       [[0.40048724],
        [0.40197116],
        [0.3974704 ],
        ...,
        [0.34718245],
        [0.34906328],
        [0.3511767 ]],

       ...,

       [[0.4006829 ],
        [0.4025823 ],
        [0.39752868],
        ...,
        [0.22952756],
        [0.22782478],
        [0.22655615]],

       [[0.40046462],
        [0.4014647 ],
        [0.39496946],
        ...,
        [0.22782329],
        [0.22655478],
        [0.22603586]],

       [[0.40022838],
        [0.40105435],
        [0.3949732 ],
        ...,
        [0.22655702],
        [0.22603816],
        [0.22620103]]], dtype=float32)

In [108]:
y_test

array([0.30596372, 0.28842555, 0.2812006 , ..., 0.17622654, 0.15276475,
       0.19080967])

In [109]:
x_test

array([[0.30596372, 0.28369085, 0.30204278, ..., 0.00105932, 0.42712599,
        0.        ],
       [0.28842555, 0.26900927, 0.2817895 , ..., 0.00105932, 0.41762742,
        0.        ],
       [0.2812006 , 0.25839058, 0.27260438, ..., 0.00105932, 0.24410641,
        0.        ],
       ...,
       [0.17622654, 0.15982672, 0.17645283, ..., 0.00105932, 0.11714905,
        0.        ],
       [0.15276475, 0.15292188, 0.16652036, ..., 0.00105932, 0.38224868,
        0.        ],
       [0.19080967, 0.19537516, 0.18643239, ..., 0.00105932, 0.36597033,
        0.        ]])

In [125]:
xx=x_test[win_length:, 1:].reshape((x_test.shape[0] - win_length, -1))

In [126]:
xx.shape

(1037, 9)

In [128]:
df_pred=pd.concat([pd.DataFrame(prediction),pd.DataFrame(xx)],axis=1)

ValueError: Must pass 2-d input. shape=(1037, 100, 1)

In [ ]:
rev_trans=scaler.inverse_transform(df_pred)

In [ ]:
rev_trans

In [ ]:
df_final=df_input[prediction.shape[0]*-1:]

In [ ]:
df_final.count()

In [ ]:
df_final['App_pred']=reve_trans[:,0]

In [ ]:
df.final()

In [ ]:
data1.head()

In [ ]:
nd_data= data1['nd'].values.reshape(-1,1)
tsd_data=data1['tsd'].values.reshape(-1,1)
ewd_data=data1['england_wales_demand'].values.reshape(-1,1)
wind_data=data1['embedded_wind_generation'].values.reshape(-1,1)
solar_data=data1['embedded_solar_generation'].values.reshape(-1,1)
windC_data=data1['embedded_wind_capacity'].values.reshape(-1,1)
solarC_data=data1['embedded_solar_capacity'].values.reshape(-1,1)
stor_data=data1['non_bm_stor'].values.reshape(-1,1)
pump_data=data1['pump_storage_pumping'].values.reshape(-1,1)
#hol_data=data1['is_holiday'].values.reshape(-1,1)
print(nd_data.shape,tsd_data.shape,ewd_data.shape,wind_data.shape,solar_data.shape,windC_data.shape,solarC_data.shape,stor_data.shape
      ,pump_data.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
nd_sca=MinMaxScaler()
tsd_sca=MinMaxScaler()
ewd_sca=MinMaxScaler()
wind_sca=MinMaxScaler()
solar_sca=MinMaxScaler()
windC_sca=MinMaxScaler()
solarC_sca=MinMaxScaler()
stor_sca=MinMaxScaler()
pump_sca=MinMaxScaler()

nd_scaler_ = nd_sca.fit_transform(nd_data)
tsd_scaler_ = tsd_sca.fit_transform(tsd_data)
ewd_scaler_ = ewd_sca.fit_transform(ewd_data)
wind_scaler_ = wind_sca.fit_transform(wind_data)
solar_scaler_ = solar_sca.fit_transform(solar_data)
windC_scaler_ = windC_sca.fit_transform(windC_data)
solarC_scaler_ = solarC_sca.fit_transform(solarC_data)
stor_scaler_ = stor_sca.fit_transform(stor_data)
pump_scaler_ = pump_sca.fit_transform(pump_data)

In [ ]:
def dataset(data,time_steps):
    x,y=[],[]
    for i in range(len(data)-time_steps):
        x.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(x),np.array(y)

time_steps=12

function to create time series data for a machine learning model

In [ ]:
from re import X
x_nd,y_nd=dataset(nd_scaler_ ,time_steps)
x_tsd,y_tsd=dataset(tsd_scaler_,time_steps)
x_ewd,y_ewd=dataset(ewd_scaler_,time_steps)
x_wnd,y_wnd=dataset(wind_scaler_,time_steps)
x_solar,y_solar=dataset(solar_scaler_,time_steps)
x_windC,y_windC=dataset(windC_scaler_,time_steps)
x_solarC,y_solarC=dataset(solarC_scaler_,time_steps)
x_stor,y_stor=dataset(stor_scaler_,time_steps)
x_pump,y_pump=dataset(pump_scaler_,time_steps)

x_ewd.shape,y_ewd.shape

Train Test Split

In [ ]:
xtrain_nd,xtest_nd,ytrain_nd,ytest_nd=tts(x_nd,y_nd,test_size=0.2,shuffle=False)
xtrain_tsd,xtest_tsd,ytrain_tsd,ytest_tsd=tts(x_tsd,y_tsd,test_size=0.2,shuffle=False)
xtrain_ewd,xtest_ewd,ytrain_ewd,ytest_ewd=tts(x_ewd,y_ewd,test_size=0.2,shuffle=False,)
xtrain_wnd,xtest_wnd,ytrain_wnd,ytest_wnd=tts(x_wnd,y_wnd,test_size=0.2,shuffle=False)
xtrain_solar,xtest_solar,ytrain_solar,ytest_solar=tts(x_solar,y_solar,test_size=0.2,shuffle=False)
x_train_windC,x_test_windC,y_train_windC,y_test_windC=tts(x_windC,y_windC,test_size=0.2,shuffle=False)
x_train_solarC,x_test_solarC,y_train_solarC,y_test_solarC=tts(x_solarC,y_solarC,test_size=0.2,shuffle=False)
x_train_stor,x_test_stor,y_train_stor,y_test_stor=tts(x_stor,y_stor,test_size=0.2,shuffle=False)
x_train_pump,x_test_pump,y_train_pump,y_test_pump=tts(x_pump,y_pump,test_size=0.2,shuffle=False)

In [ ]:
xtrain_nd.shape

In [ ]:
data1.columns

In [ ]:
X_train=np.concatenate((xtrain_nd,xtrain_tsd,xtrain_ewd,xtrain_wnd,xtrain_solar,x_train_windC,x_train_solarC,x_train_stor,x_train_pump),axis=2)
y_train=np.concatenate((ytest_nd,ytest_tsd,ytest_ewd),axis=1)
X_test=np.concatenate((xtrain_nd,xtrain_tsd,xtrain_ewd,xtrain_wnd,xtrain_solar,x_train_windC,x_train_solarC,x_train_stor,x_train_pump),axis=2)
y_test=np.concatenate((ytest_nd,ytest_tsd,ytest_ewd),axis=1)

In [ ]:
model=Sequential()
model.add(LSTM(units=128,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=64,return_sequences=True,activation='relu'))
model.add(LSTM(units=32,return_sequences=True,activation='relu'))
model.add(LSTM(units=16,return_sequences=True,activation='relu'))
model.add(LSTM(units=8,return_sequences=True,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(54,activation='relu'))
model.add(Dense(28,activation='relu'))
model.add(Dense(12,activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(units=3,activation='linear'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])